# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): (впишите свои фамилию и имя)

### Общая информация
Дата выдачи: 09.04.2020

Дедлайн: 24.04.2020 23:59 MSK

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [0]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-03-31 09:58:01--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-03-31 09:58:01--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucae286d974c1a93364efc4c15b2.dl.dropboxusercontent.com/cd/0/inline/A08xSoWyLqzqkg3IZST5TsmXcFjlXWpm4igRmrNYI9-NdkryhCoDBxr_b-aRGBPHEeVY1bQo9aN9aI788NmNqV_nYufmH0n2rupAIUYBWOcPqpAuB_dCDIlXLNoMVzPcPbc/file# [following]
--2020-03-31 09:58:02--  https://ucae286d974c1a93364efc4c15b2.dl.dropboxusercontent.com/cd/0/inline/A08xSoWyLqzqkg3IZST5TsmXcFjlXWpm4igRmrNYI9-NdkryhCoDBxr_b-aRGBPHEeVY1bQo9aN9aI788NmNqV_nYufmH0n2rup

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [2]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [3]:
data.shape

(30000, 4)

In [4]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [7]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [8]:
# Your code here

In [9]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [10]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов
 - Для каждого примера посчитать, сколько раз каждое слово из словаря в нём встречается


В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите 10000 самых частых слов из `title` и `description`, отсортируйте их по убыванию частотности.

In [19]:
# Your code here
bow_vocabulary = ...

In [49]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [21]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из bow_vocabulary
    указано количество его употреблений
    """ 
    # Your code here

In [22]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [34]:
def items_to_bow(items: np.array, use_title=False) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте строить bow только из description товара

    # Your code here

In [35]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

100%|██████████| 1/1 [00:00<00:00, 163.85it/s]


In [25]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

100%|██████████| 9000/9000 [00:32<00:00, 279.77it/s]


### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.

**Подсказка: рассмотрите использование sparse-матриц - в них хранятся только ненулевые элементы.** `scipy.sparse.csr_matrix(X_train_bow)`

In [30]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

bow_model_lr = LogisticRegression(max_iter=100)
bow_model_lr.fit(X_train_bow_sparsed, y_train)
acc_lr_bow = accuracy_score(bow_model_lr.predict(X_test_bow_sparsed), y_test)
print(acc_lr_bow)

assert acc_lr_bow > 0.7

0.7011111111111111


/Users/vkokhtev/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [50]:
from sklearn.svm import LinearSVC

bow_model_svc = LinearSVC(max_iter=70)
bow_model_svc.fit(X_train_bow, y_train)
acc_svc_bow = accuracy_score(bow_model_svc.predict(X_test_bow_sparsed), y_test)
print(acc_svc_bow)

assert acc_svc_bow > 0.68

0.6847777777777778


/Users/vkokhtev/Library/Python/3.7/lib/python/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Модификация признаков (0.5 балла)

Постройте признаковое пространство на конкатенации (через пробел) `title` и `description`.

In [36]:
# Your code here

100%|██████████| 9000/9000 [00:30<00:00, 293.76it/s]


### mystem (1.5) балла

Как можно заметить, в текстах одни и те же слова могут быть в разных падежах, а соответственно в bow это будут разные признаки. 
Чтобы исправить это, можно лемматизировать слова - с помощью библиотеки Mystem.

In [32]:
!pip3 install pymystem3

Defaulting to user installation because normal site-packages is not writeable


In [47]:
from pymystem3 import Mystem

m = Mystem()
m.lemmatize("Здравствуйте, ваше благородие")

['здравствовать', ', ', 'ваш', ' ', 'благородие', '\n']

__Задание:__ 
* Лемматезируйте `title` и `description` с помощью mystem
* Gостройте BoW-представление
* Обучите на нём логистическую регрессию и SVC.

In [52]:
# Your code here

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

Давайте для простоты считать один tf-idf для title и description. Для каждого слова из bow_vocabulary нужно посчитать:
1. Сколько раз оно встретилось в title и description во всём X_train
2. В тексте скольких товаров встретилось это слово

In [54]:
count_arr = ...#

In [55]:
def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из bow_vocabulary указан tf-idf
    """

    # Your code here

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [0]:
# Your code here

### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [0]:
# Your code here

### Word Vectors (1.5 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [0]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-03-31 16:45:00--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-03-31 16:45:00--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf74acabb02d88214319f005183.dl.dropboxusercontent.com/cd/0/inline/A087WRc_C-FkDvCVbFXdMKdMyQWRCexUAvk6YIRSoEau8Wq5ksdlNc1cSEiPXmVgH-PCYXTEX9bRxCxm-rQzL6jX_HpbKnwGLF_pva7_h6eR9w/file# [following]
--2020-03-31 16:45:01--  https://ucf74acabb02d88214319f005183.dl.dropboxusercontent.com/cd/0/inline/A087WRc_C-FkDvCVbFXdMKdMyQWRCexUAvk6YIRSoEau8Wq5ksdlNc1cSEiPXmVgH-PCYXTEX9bRxCxm-rQzL6jX_HpbKnwGLF_pva7_h6eR9w/file
Resolving ucf74acabb02d88

In [0]:
!tar -xzfv ru.tar.gz

In [0]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

In [0]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [0]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

__Задание:__ 
* Обучите логистическую регрессию и SVM на новых признаках
* Оцените качество (accuracy_score)

### Что дальше?

Решение каждого пункта 2 балла:



1. N-Gram модели текстовой классификации
    * Признаки - mystem
    * n-gramm'ы - несколько слов подряд объединяются в один токен
    * Модели - только логистическая регрессия
    * Настоятельно рекомендуется использовать sparse-матрицы
    

2. Использовать Vowpal Wabbit вместо sklearn
    * Признаки - обычный BoW title+description 
    * Главный вызов - заставить работать библиотеку и подготовить признаки

3. Другие способы лемматизации (pymorphy2, spaCy)


* Снабжайте код пояснениями;
* Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.